# Clinical trial optimization 
This cookbook will serve as a prototype of the trial optimization feature.

## Loading dependencies

In [ ]:
# Cookbook specifics imports
import matplotlib.pyplot as plt
from modAL.models import BayesianOptimizer
from modAL.acquisition import max_EI
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.model_selection import train_test_split

In [ ]:
# initializing the learner
kernel = Matern(length_scale=1.0)
regressor = GaussianProcessRegressor(kernel=kernel)

d = 8
n, n_training = 10000, 100
a = 0.3 * np.ones((d,))
X_full = np.random.randn(n, d)
y_full = 1 - np.sum((X_full - a) ** 2, axis=1)
i_max = np.argmax(y_full)


def fitness(x):
    return 1 - np.sum((x - a) ** 2)


optimizer = BayesianOptimizer(
    estimator=regressor,
    # X_training     = X_initial, y_training = y_initial,
    X_training=X_full[:n_training, :],
    y_training=y_full[:n_training],
    query_strategy=max_EI,
)
X_max, y_max = optimizer.get_max()
print(f"Training set optimal value: {y_max}")
print(f"fitness: {fitness(X_max)}")

In [ ]:
num_queries = 5
queried = []
for n_query in range(num_queries):
    query_idx, query_inst = optimizer.query(X_full)
    queried.append(query_idx[0])
    y = fitness(X_full[query_idx])
    optimizer.teach(X_full[query_idx, :].reshape(1, -1), np.array([y]))
    # X_max, y_max = optimizer.get_max()
    print(f"iteration = {n_query+1}, queried index: {query_idx}, fitness = {y:.3g}")
X_max, y_max = optimizer.get_max()
print(f"X_max={X_max}, y_max={y_max:.3g}")

In [ ]:
x = X_full[:1000, :]
y_pred, y_std = optimizer.predict(x, return_std=True)
plt.figure()
plt.scatter(x[:, 0], y_pred, s=10, c="b", zorder=-1)
plt.errorbar(x[:, 0], y_pred, yerr=y_std, fmt=".", zorder=0, label="GP regressor")
plt.scatter(
    optimizer.X_training[:, 0],
    optimizer.y_training,
    c="k",
    s=10,
    label="Training",
    zorder=1,
)
plt.scatter(X_full[queried, 0], y_full[queried], c="r", s=10, label="Queried", zorder=2)
plt.scatter(
    X_full[i_max, 0],
    y_full[i_max],
    s=100,
    marker="*",
    zorder=1.5,
    c="g",
    label="full design best",
)
plt.legend()